<a href="https://colab.research.google.com/github/sanha-hwang/ocular_disease/blob/main/ocular_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1>Project4_level1: 목표   



1. 캐글에서 안구질환 인식 데이터를 받아 안구질환을 진단하는 모델을 만들어보자    


2. 5000명의 환자의 안저 사진 데이터가 있고 라벨로는
일반 : N, 당뇨병 : D, 녹내장 : G, 백내장 : C, 연령 관련 황반변성 : A
고혈압 : H, 병적 근시 : M, 기타 질병 / 이상 : O

Normal (N),
Diabetes (D),
Glaucoma (G),
Cataract (C),
Age related Macular Degeneration (A),
Hypertension (H),
Pathological Myopia (M),
Other diseases/abnormalities (O)

3. 이미지 분류 잘하는 모델 만들기 목표 80프로 이상

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [49]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow
from tensorflow import keras

base_dir = '/content/drive/MyDrive/Input'
train_dir = os.path.join(base_dir,'preprocessed_images')

dataset_path = os.path.join(base_dir,'full_df.csv')


In [107]:
#시도

try_dir =os.path.join(base_dir,'ocular-disease-recognition-odir-5k/ODIR-5K')
try_dataset_dir = os.path.join(try_dir,'data.xlsx')
try_dataset = pd.read_excel(try_dataset_dir)
try_dataset.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1
3,3,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1
4,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1


In [110]:
dataset = pd.read_csv(dataset_path)
pre_data = dataset.loc[:,['ID','Patient Age','Patient Sex','filename','target','labels']]
target_info = ['N','D', 'G', 'C', 'A', 'H', 'M', 'O']

In [111]:
dataset.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [108]:
pre_data.head()

,ID,Patient Age,Patient Sex,filename,target,labels
0,0,69,0,0_right.jpg,"[1, 0, 0, 0, 0, 0, 0, 0]",['N']
1,1,57,0,1_right.jpg,"[1, 0, 0, 0, 0, 0, 0, 0]",['N']
2,2,42,0,2_right.jpg,"[0, 1, 0, 0, 0, 0, 0, 0]",['D']
3,4,53,0,4_right.jpg,"[0, 1, 0, 0, 0, 0, 0, 0]",['D']
4,5,50,0,5_right.jpg,"[0, 1, 0, 0, 0, 0, 0, 0]",['D']


In [52]:
# male : 0, female : 1
def gender(text):
  text = text.lower()
  if 'male' in text:
    return 0
  elif 'female' in text:
    return 1

pre_data['Patient Sex'] = pre_data['Patient Sex'].apply(gender)

In [53]:
target_info

['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

In [57]:
def labeling(str):
  if str == "['N']" :
    return [1,0,0,0,0,0,0,0]
  elif str == "['D']":
    return [0,1,0,0,0,0,0,0]
  elif str == "['G']":
    return [0,0,1,0,0,0,0,0]
  elif str == "['C']":
    return [0,0,0,1,0,0,0,0]
  elif str == "['A']":
    return [0,0,0,0,1,0,0,0]
  elif str == "['H']":
    return [0,0,0,0,0,1,0,0]
  elif str =="['M']":
    return [0,0,0,0,0,0,1,0]
  else:
    return [0,0,0,0,0,0,0,1]
  

In [59]:
import random
image_size=224
dataset = []

for i in tqdm(range(len(pre_data))):
  img =pre_data.loc[i,['filename']].values[0]
  image_path = os.path.join(train_dir,img)
  try:
    image = cv2.imread(image_path,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(image_size,image_size))
  except:
    continue
  label = labeling(pre_data.loc[i,['labels']][0])
  dataset.append([np.array(image),np.array(label)])


random.shuffle(dataset)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
100%|██████████| 6392/6392 [55:17<00:00,  1.93it/s]


In [90]:
#Step. model define 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.applications import vgg16, resnet50, mobilenet, inception_v3

Num_classes = 8 # N, D, G, C, A, H, M, O

channels = 3 # 입력 이미지의 차원수 : RBG

#학습된 네트워크 Resnet50
image_size =224
resnet_pooling_average = 'avg'
dense_layer_activation = 'softmax'
objective_function = 'categorical_crossentropy'

#출력 metrics
loss_metrics = ['accuracy']

#early_stop
num_epochs = 20
early_stop_patience = 5

step_per_epoch_training = 10
step_per_epoch_validation = 10 

batch_size_training = 100
batch_size_validation = 100

batch_size_testing = 1

#Step model making
vgg_model = vgg16.VGG16(weights='imagenet',  include_top = False, pooling = resnet_pooling_average)
inception_model = inception_v3.InceptionV3(weights='imagenet',  include_top = False, pooling = resnet_pooling_average)
resnet_model = resnet50.ResNet50(weights='imagenet',  include_top = False, pooling = resnet_pooling_average)
mobilenet_model = mobilenet.MobileNet(weights='imagenet',  include_top = False, pooling = resnet_pooling_average)


model_resnet = Sequential()
model_resnet.add(resnet_model)
model_resnet.add(Dense(Num_classes, activation = dense_layer_activation))

model_vgg =Sequential()
model_vgg.add(vgg_model)
model_vgg.add(Dense(Num_classes, activation = dense_layer_activation))

model_inception =Sequential()
model_inception.add(inception_model)
model_inception.add(Dense(Num_classes, activation = dense_layer_activation))

model_mobile =Sequential()
model_mobile.add(mobilenet_model)
model_mobile.add(Dense(Num_classes, activation = dense_layer_activation))


for layer in model.layers:
  layer.trainable = False

#Step model compiler

from tensorflow.keras import optimizers

model_resnet.compile(optimizer = 'adam', loss = objective_function, metrics = loss_metrics)
model_vgg.compile(optimizer = 'adam', loss = objective_function, metrics = loss_metrics)
model_inception.compile(optimizer = 'adam', loss = objective_function, metrics = loss_metrics)
model_mobile.compile(optimizer = 'adam', loss = objective_function, metrics = loss_metrics)

#check point save 

filename_res = 'res_{epoch:02d}-{val_loss:.5f}.h5'
checkpoint_res = ModelCheckpoint(filename_res,             
                             monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True,  # 가장 best 값만 저장합니다
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )

filename_vgg = 'vgg_{epoch:02d}-{val_loss:.5f}.h5'
checkpoint_vgg = ModelCheckpoint(filename_vgg,             
                             monitor='val_loss',   
                             verbose=1,            
                             save_best_only=True,  
                             mode='auto'           
                            )

filename_inception = 'inception_{epoch:02d}-{val_loss:.5f}.h5'
checkpoint_inception = ModelCheckpoint(filename_inception,             
                             monitor='val_loss',   
                             verbose=1,            
                             save_best_only=True,  
                             mode='auto'          
                            )

filename_mobile = 'mobile_{epoch:02d}-{val_loss:.5f}.h5'
checkpoint_mobile = ModelCheckpoint(filename_mobile,             
                             monitor='val_loss',   
                             verbose=1,            
                             save_best_only=True,  
                             mode='auto'           
                            )
#earlystop
earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=early_stop_patience)



In [91]:
model_resnet.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_13 (Dense)             (None, 8)                 16392     
Total params: 23,604,104
Trainable params: 23,550,984
Non-trainable params: 53,120
_________________________________________________________________


In [92]:
model_vgg.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 4104      
Total params: 14,718,792
Trainable params: 14,718,792
Non-trainable params: 0
_________________________________________________________________


In [93]:
model_inception.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
dense_15 (Dense)             (None, 8)                 16392     
Total params: 21,819,176
Trainable params: 21,784,744
Non-trainable params: 34,432
_________________________________________________________________


In [94]:
model_mobile.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Function (None, 1024)              3228864   
_________________________________________________________________
dense_16 (Dense)             (None, 8)                 8200      
Total params: 3,237,064
Trainable params: 3,215,176
Non-trainable params: 21,888
_________________________________________________________________


In [95]:
from sklearn.model_selection import train_test_split
from keras.models import load_model

x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3)
y = np.array([i[1] for i in dataset])

x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.2, random_state= 42)


In [99]:
history_resnet = model_resnet.fit(x_train,y_train,batch_size=32,epochs=num_epochs,validation_data=(x_test,y_test),
                    verbose=1 ,callbacks=[checkpoint_res,earlystop])

loss,accuracy = model_resnet.evaluate(x_test,y_test)
print("loss:",loss)
print("Accuracy:",accuracy)

Epoch 1/20
160/160 [==============================] - 23s 113ms/step - loss: 1.5058 - accuracy: 0.4471 - val_loss: 1.7179 - val_accuracy: 0.4699

Epoch 00001: val_loss improved from inf to 1.71792, saving model to res_01-1.71792.h5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/20
160/160 [==============================] - 17s 109ms/step - loss: 1.2900 - accuracy: 0.5138 - val_loss: 1.3133 - val_accuracy: 0.4918

Epoch 00002: val_loss improved from 1.71792 to 1.31326, saving model to res_02-1.31326.h5
Epoch 3/20
160/160 [==============================] - 17s 109ms/step - loss: 1.1876 - accuracy: 0.5470 - val_loss: 1.2541 - val_accuracy: 0.5418

Epoch 00003: val_loss improved from 1.31326 to 1.25411, saving model to res_03-1.25411.h5
Epoch 4/20
160/160 [==============================] - 17s 109ms/step - loss: 1.1016 - accuracy: 0.5832 - val_loss: 3.8864 - val_accuracy: 0.2361

Epoch 00004: val_loss did not improve from 1.25411
Epoch 5/20
160/160 [==============================] - 17s 109ms/step - loss: 1.0415 - accuracy: 0.6077 - val_loss: 1.9772 - val_accuracy: 0.3417

Epoch 00005: val_loss did not improve from 1.25411
Epoch 6/20
160/160 [==============================] - 17s 108ms/step - loss: 0.9509 - accuracy: 0.6482 - val_loss: 1.6031 - val_accura

In [100]:
history_vgg = model_vgg.fit(x_train,y_train,batch_size=32,epochs=num_epochs,validation_data=(x_test,y_test),
                    verbose=1 ,callbacks=[checkpoint_vgg,earlystop])

loss,accuracy = model_vgg.evaluate(x_test,y_test)
print("loss:",loss)
print("Accuracy:",accuracy)

Epoch 1/20
160/160 [==============================] - 21s 128ms/step - loss: 6.9721 - accuracy: 0.4178 - val_loss: 1.6048 - val_accuracy: 0.4566

Epoch 00001: val_loss improved from inf to 1.60480, saving model to vgg_01-1.60480.h5
Epoch 2/20
160/160 [==============================] - 20s 127ms/step - loss: 1.6120 - accuracy: 0.4477 - val_loss: 1.6181 - val_accuracy: 0.4566

Epoch 00002: val_loss did not improve from 1.60480
Epoch 3/20
160/160 [==============================] - 20s 127ms/step - loss: 1.6045 - accuracy: 0.4473 - val_loss: 1.5954 - val_accuracy: 0.4566

Epoch 00003: val_loss improved from 1.60480 to 1.59536, saving model to vgg_03-1.59536.h5
Epoch 4/20
160/160 [==============================] - 20s 127ms/step - loss: 1.6119 - accuracy: 0.4477 - val_loss: 1.6045 - val_accuracy: 0.4566

Epoch 00004: val_loss did not improve from 1.59536
Epoch 5/20
160/160 [==============================] - 20s 127ms/step - loss: 1.6036 - accuracy: 0.4477 - val_loss: 1.5917 - val_accuracy: 

In [97]:
history_inception = model_inception.fit(x_train,y_train,batch_size=32,epochs=num_epochs,validation_data=(x_test,y_test),
                    verbose=1 ,callbacks=[checkpoint_inception,earlystop])

loss,accuracy = model_inception.evaluate(x_test,y_test)
print("loss:",loss)
print("Accuracy:",accuracy)

Epoch 1/20
160/160 [==============================] - 18s 110ms/step - loss: 1.3291 - accuracy: 0.5026 - val_loss: 3.3172 - val_accuracy: 0.2635

Epoch 00001: val_loss improved from inf to 3.31717, saving model to inception_01-3.31717.h5
Epoch 2/20
160/160 [==============================] - 15s 94ms/step - loss: 1.1429 - accuracy: 0.5684 - val_loss: 1.3244 - val_accuracy: 0.5324

Epoch 00002: val_loss improved from 3.31717 to 1.32437, saving model to inception_02-1.32437.h5
Epoch 3/20
160/160 [==============================] - 15s 94ms/step - loss: 1.0456 - accuracy: 0.6063 - val_loss: 2.1118 - val_accuracy: 0.5066

Epoch 00003: val_loss did not improve from 1.32437
Epoch 4/20
160/160 [==============================] - 15s 94ms/step - loss: 0.9715 - accuracy: 0.6405 - val_loss: 1.2874 - val_accuracy: 0.5543

Epoch 00004: val_loss improved from 1.32437 to 1.28741, saving model to inception_04-1.28741.h5
Epoch 5/20
160/160 [==============================] - 15s 94ms/step - loss: 0.8758 -

In [98]:
history_mobile = model_mobile.fit(x_train,y_train,batch_size=32,epochs=num_epochs,validation_data=(x_test,y_test),
                    verbose=1 ,callbacks=[checkpoint_mobile,earlystop])

loss,accuracy = model_mobile.evaluate(x_test,y_test)
print("loss:",loss)
print("Accuracy:",accuracy)

Epoch 1/20
160/160 [==============================] - 20s 113ms/step - loss: 1.3532 - accuracy: 0.5056 - val_loss: 1.5292 - val_accuracy: 0.4902

Epoch 00001: val_loss improved from inf to 1.52923, saving model to mobile_01-1.52923.h5
Epoch 2/20
160/160 [==============================] - 17s 108ms/step - loss: 1.0282 - accuracy: 0.6096 - val_loss: 1.8013 - val_accuracy: 0.5614

Epoch 00002: val_loss did not improve from 1.52923
Epoch 3/20
160/160 [==============================] - 17s 108ms/step - loss: 0.8389 - accuracy: 0.6824 - val_loss: 2.3863 - val_accuracy: 0.4973

Epoch 00003: val_loss did not improve from 1.52923
Epoch 4/20
160/160 [==============================] - 17s 108ms/step - loss: 0.6924 - accuracy: 0.7324 - val_loss: 1.4941 - val_accuracy: 0.5723

Epoch 00004: val_loss improved from 1.52923 to 1.49409, saving model to mobile_04-1.49409.h5
Epoch 5/20
160/160 [==============================] - 17s 108ms/step - loss: 0.5776 - accuracy: 0.7905 - val_loss: 1.7342 - val_accu

In [101]:
model_resnet.save('model_resnet.h5')
model_vgg.save('model_vgg.h5')
model_inception.save('model_inception.h5')
model_mobile.save('model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [103]:
y_pred = model_mobile.predict_classes(x_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [106]:
y_pred

array([0, 0, 0, ..., 0, 4, 0])

In [ ]:
#번외편 이미지 한번 눈으로 확인해보기
%matplotlib inline

# import matplotlib.image as mpimg
# import matplotlib.pyplot as plt

# nrows, ncols = 4, 4
# pic_index = 0

# fig = plt.gcf()
# fig.set_size_inches(ncols*3, nrows*3)

# pic_index+=8

# next_ocular_pix = [os.path.join(train_dir, fname)
#                 for fname in train_fname[pic_index-8:pic_index]]

# for i, img_path in enumerate(next_ocular_pix):
#   sp = plt.subplot(nrows, ncols, i + 1)
#   sp.axis('Off')
#   img = mpimg.imread(img_path)
#   plt.imshow(img)

# plt.show()

In [ ]:
# Step input image resize & augmentation

# from keras.applications.resnet50 import preprocess_input
# from keras.preprocessing.image import ImageDataGenerator

# data_generator = ImageDataGenerator(
#     rescale = 1.0/255.,
#     preprocessing_function = preprocess_input,
#     featurewise_center = True,
#     featurewise_std_normalization = True,
#     rotation_state = 20,
#     width_sift_range = 0.2,
#     height_shift_range = 0.2,
#     horizontal_flip = True)

# train_generator = data_generator.flow_from_directory(trainset_dir,
#                                                      batch_size = batch_size_training,
#                                                      class_mode = 'categorical')

# validation_gen = ImageDataGenerator( rescale = 1.0/255.)
# validation_generator = validation_gen.flow_from_directory(validation_dir,
#                                                          batch_size = batch_size_training,
#                                                          class_mode = 'categorical')